In [1]:
import torch

# checkpoint = torch.load('/home/surya/Downloads/fpn_dat_t_80k.pth')
checkpoint = torch.load('/home/surya/Downloads/mrcn_dat_t_1x.pth')

from collections import OrderedDict
new_state_dict = OrderedDict()

for k,v in checkpoint['state_dict'].items():
    if('backbone' in k):
        if(k.startswith("backbone.norms.")):
            pass
        else:
            new_key = k.replace("backbone.", "")
            new_state_dict[new_key] = v
torch.save(new_state_dict, '/home/surya/Downloads/mrcn_dat_t_backbone.pth')

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load pretrained model
checkpoint = torch.load('/home/surya/Downloads/mrcn_dat_t_backbone.pth')

from dat import DAT
model = DAT()
model.eval();
model.to(device);
model.load_state_dict(checkpoint, strict=False)

_IncompatibleKeys(missing_keys=['cls_norm.norm.weight', 'cls_norm.norm.bias', 'cls_head.weight', 'cls_head.bias'], unexpected_keys=[])

In [3]:
new_state_dict_keyset = set(new_state_dict.keys())
model_state_dict_keyset = set(model.state_dict().keys())
model_state_dict_keyset.difference(new_state_dict_keyset)

{'cls_head.bias',
 'cls_head.weight',
 'cls_norm.norm.bias',
 'cls_norm.norm.weight'}